# How to use a team of Mosaia agents locally using AG2 Autogen

This Jupyter Notebook outlines the steps for importing a group of agents from Mosaia to be used in a local implementation of AG2 Autogen.

## Requirement

Requires Python version 3.11 and `autogen`, which can be installed by the following command:

In [ ]:
%pip install autogen "flaml[automl]"

## Step 1: retrieve the AG2 autogen config json from Mosaia
Click the following link https://www.mosaia.ai/user/aaron/team/project-ideation-team?tab=agents in a browser. Click the blue `Export` on the top and then `autogen`. This will download an autogen config json to be used in this notebook. Make sure to have this file in the same directory location as this notebook.

## Step 2: import the required dependencies from AG2 autogen

In [ ]:
import autogen
from autogen.agentchat.contrib.agent_builder import AgentBuilder
from autogen import (
    UserProxyAgent,
    GroupChat,
    GroupChatManager
)

## Step 2: prepare configuration and some useful functions
Prepare a `config_file_or_env` for assistant agent to limit the choice of LLM you want to use in this task. This config can be a path of json file or a name of environment variable. A `default_llm_config` is also required for initialize the specific config of LLMs like seed, temperature, etc...

In [ ]:
config_file_or_env = "OAI_CONFIG_LIST"
llm_config = {"temperature": 0}
config_list = autogen.config_list_from_json(config_file_or_env, filter_dict={"model": ["gpt-4o-mini"]})

## Step 3: create a AgentBuilder
Create a `AgentBuilder` with the specified `config_path_or_env`.

In [ ]:
builder = AgentBuilder(config_file_or_env=config_file_or_env)

## Step 4: create agents using the autogen schema from Mosaia

reference the path to the autogen config json file on your local machine in the `builder.load` function

In [ ]:
agent_list, agent_configs = builder.load(
    "./project-ideation-team_autogen_export.json"
)

## Step 4: create a group chat task runner
Using Autogen's `UserProxyAgent`, `GroupChat` and `GroupChatManager`. Create a task runner for the hydrated team of agents to work on an assigned task given by the `UserProxyAgent`.

In [ ]:
def start_task(execution_task: str, agent_list: list, coding=True):
    user_proxy_system_message = "A human admin proxy for interacting with multi-agent systems"
    user_proxy = UserProxyAgent(
        name="user",
        code_execution_config={
            "work_dir": "coding",
            "use_docker": False,
        },
        system_message=user_proxy_system_message,
        is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
        human_input_mode="NEVER",
        default_auto_reply="TERMINATE",
        max_consecutive_auto_reply=1
    )

    agent_list.append(user_proxy)

    group_chat = GroupChat(
        agents=agent_list,
        messages=[],
        max_round=12,
        allow_repeat_speaker=agent_list[:-1] if coding is True else agent_list,
    )
    manager = GroupChatManager(
        groupchat=group_chat,
        llm_config={"config_list": config_list, **llm_config},
    )
    chat_result = user_proxy.initiate_chat(manager, message=execution_task)

    return chat_result

## Step 5: execute task
Let's run a task with our group of agents using the task runner

In [ ]:
chat_result = start_task(
    execution_task="Give me some ideas for a viable AI startup",
    agent_list=agent_list,
)

## Step 6: view task results

In [ ]:
chat_result

## Step 7: clear the agent builder
Clear the cache of the agent builder to hydrate a new team of agents

In [ ]:
builder.clear_all_agents()